# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import geometric_sampling as gs
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri, default_converter
from rpy2.robjects.conversion import localconverter
from package_sampling.utils import inclusion_probabilities

/home/divar/projects/graphical-sampling/.venv/lib/python3.12/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "PWD" redefined by R and overriding existing variable. Current: "/home/divar/projects/graphical-sampling", R: "/home/divar/projects/graphical-sampling/Simulations"
  warnings.warn(
/home/divar/projects/graphical-sampling/.venv/lib/python3.12/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "LD_LIBRARY_PATH" redefined by R and overriding existing variable. Current: "/usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server", R: "/usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server:/usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server"
  warnings.warn(
/home/divar/projects/graphical-sampling/.venv/lib/python3.12/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "R_LIBS_SITE" redefined by R and overriding existing va

In [3]:
%load_ext rpy2.ipython

In [4]:
%%R

library(WaveSampling)
library(sampling)
library(BalancedSampling)


Loading required package: Matrix


# Moran and Local Balance Score

In [5]:
def score_all_samples_moran_lb(coords, probs, sample_indices_list):
    """
    coords          : an (N×2)-array of spatial coordinates
    probs           : length-N array of inclusion probabilities
    sample_indices_list : list of length-n integer numpy arrays (0-based indices)

    Returns an (S×2) numpy array of [IB, SBLB] for each of the S samples.
    """

    # Convert Python list of numpy arrays into an R list of integer vectors
    #   * add +1 because R is 1-based
    r_sample_list = ro.ListVector({
        str(i+1): ro.IntVector(sample_idx.astype(int) + 1)
        for i, sample_idx in enumerate(sample_indices_list)
    })

    with localconverter(default_converter + numpy2ri.converter):
        ro.globalenv['coords'] = coords
        ro.globalenv['probs'] = probs
        # Precompute W once
        ro.r("""
            W0 <- wpik(coords, probs)
            W <- W0 - diag(diag(W0))
            diag(W) <- 0
        """)
        ro.globalenv['samples'] = r_sample_list

        # Define an R function that loops over all samples
        ro.r("""
            score_samples <- function(W, probs, coords, samples_list) {
              S <- length(samples_list)
              IBs   <- numeric(S)
              SBLBs <- numeric(S)

              for (i in seq_len(S)) {
                samp_idx <- samples_list[[i]]
                mask <- integer(length(probs))
                mask[samp_idx] <- 1

                IBs[i]   <- tryCatch(IB(W, mask),        error = function(e) Inf)
                SBLBs[i] <- tryCatch(sblb(probs, coords, samp_idx), error = function(e) Inf)
              }
              # return as a 2-column matrix
              cbind(IB = IBs, SBLB = SBLBs)
            }
        """)

        # Call it once
        result = ro.r("score_samples(W, probs, coords, samples)")
        # result comes back as an R matrix  S×2

    # Turn it into an (S×2) numpy array
    with localconverter(default_converter + numpy2ri.converter):
        np_result = np.array(result)
    return np_result[:, 0], np_result[:, 1]


# Loading Population

In [6]:
import os
import glob
import numpy as np

DATA_DIR = "../data_samples/coords_probs"
csv_paths = glob.glob(os.path.join(DATA_DIR, "*.csv"))

coords_dict = {}
probs_dict = {}

for fp in csv_paths:
    print(fp)
    if 'swiss' not in fp:
        name = os.path.splitext(os.path.basename(fp))[0]  # e.g. 'meuse_eq_n=5'
        data = np.loadtxt(fp, delimiter=",", skiprows=1)
        coords = data[:, :2]
        probs  = data[:, -1]
        print(name)
        
        parts = name.split('_')
        if len(parts) < 2:
            print(f"Filename pattern unexpected: {name}")
            continue
        
        coord_name = parts[0]
        prob_name  = parts[1]
        # Join any extra parts (like n=5)
        extra = "_".join(parts[2:]) if len(parts) > 2 else "" 
        
        # Normalize values
        coord_name = 'cluster' if coord_name == 'clust' else coord_name
        prob_name = 'equal' if prob_name == 'eq' else ('unequal' if prob_name == 'uneq' else prob_name)
        
        # Build final key (including n=5 if present)
        key = f"{coord_name}_{prob_name}"
        if extra:
            key += f"_{extra}"

        coords_dict[key] = coords
        probs_dict[key] = probs  # or use probs_dict[key] = {'coords': coords, 'probs': probs}

print(coords_dict.keys())
print(probs_dict.keys())
# Access example: probs_dict['meuse_equal_n=5']

../data_samples/coords_probs/swiss_uneq_n=5.csv
../data_samples/coords_probs/swiss_uneq_n=10.csv
../data_samples/coords_probs/grid_uneq.csv
grid_uneq
../data_samples/coords_probs/meuse_eq.csv
meuse_eq
../data_samples/coords_probs/meuse_eq_n=5.csv
meuse_eq_n=5
../data_samples/coords_probs/swiss_eq_n=8.csv
../data_samples/coords_probs/swiss_eq_n=20.csv
../data_samples/coords_probs/swiss_eq_n=10.csv
../data_samples/coords_probs/swiss_eq_n=4.csv
../data_samples/coords_probs/meuse_eq_n=10.csv
meuse_eq_n=10
../data_samples/coords_probs/clust_eq.csv
clust_eq
../data_samples/coords_probs/meuse_uneq_n=5.csv
meuse_uneq_n=5
../data_samples/coords_probs/swiss_uneq_n=20.csv
../data_samples/coords_probs/meuse_uneq_n=10.csv
meuse_uneq_n=10
../data_samples/coords_probs/swiss_eq_n=5.csv
../data_samples/coords_probs/swiss_e222q.csv
../data_samples/coords_probs/clust_uneq.csv
clust_uneq
../data_samples/coords_probs/swiss_uneq_n=8.csv
../data_samples/coords_probs/meuse_uneq.csv
meuse_uneq
../data_samples/

# Evaluation Function

In [49]:
def top_n_records(df: pd.DataFrame,
                  name_cols: list,
                  sort_col: str = 'exp_moran',
                  k: int = 10,
                  smallest: bool = True) -> pd.DataFrame:

    df_copy = df.copy()

    df_copy['name'] = df_copy[name_cols].astype(str).agg(' - '.join, axis=1)

    if smallest:
        return df_copy.nsmallest(k, sort_col)[['name', sort_col]].reset_index(drop=True)
    return df_copy.nlargest(k, sort_col)[['name', sort_col]].reset_index(drop=True)

In [53]:
def evaluate(coords_dict, probs_dict, n_values, zone_list,
             sort_method_list, zonal_sort_list,
             tolerance=5, split_size=1e-3, zone_mode_list=("sweep",)):
    rows = []
    combos = itertools.product(n_values, zone_list, sort_method_list, zonal_sort_list, zone_mode_list)

    for n, zones, sort_method, zonal_sort, zone_mode in combos:
        for coord_name, coords in coords_dict.items():
            prob_container = probs_dict.get(coord_name, None)

            if prob_container is None:
                raise KeyError(f"No probs for coord set '{coord_name}'")

            # normalize to dict
            if isinstance(prob_container, np.ndarray):
                prob_iter = { "default": prob_container }
            else:
                prob_iter = prob_container  # dict-like

            for prob_name, probs in prob_iter.items():
                mod_probs = inclusion_probabilities(probs, n=n)

                kss = gs.sampling.KMeansSpatialSamplingSimple(
                    coords, mod_probs,
                    n=n,
                    n_zones=zones if isinstance(zones, tuple) else (zones, zones),
                    tolerance=tolerance,
                    split_size=split_size,
                    zone_mode=zone_mode,
                    sort_method=sort_method,
                    zonal_sort=zonal_sort,
                    max_missed_samples=1
                )

                density_exp = np.round(kss.expected_score(), 4)
                density_var = np.round(kss.var_score(), 4)
                # ... other metrics

                rows.append({
                    "coord": coord_name,
                    "prob": prob_name,
                    "n": n,
                    "zones": zones,
                    "zone_mode": zone_mode,
                    "sort_method": sort_method,
                    "zonal_sort": zonal_sort,
                    "exp_density": density_exp,
                    "var_density": density_var,
                })

    return pd.DataFrame(rows)


In [54]:
df = evaluate(
    coords_dict,
    probs_dict,
    n_values=[4],
    # zone_list=[(1, 1), (2, 2), (3, 3)],
    zone_list=[2, 3, 4],
    sort_method_list=['distance_0', 'projection'],
    zonal_sort_list=[None, 'distance_0', 'projection'],
    tolerance=5,
    split_size=1e-3,
)

TypeError: PopulationSimple._normalize() takes 1 positional argument but 2 were given

In [ ]:
summary = df.pivot_table(
    columns=['zones', 'zonal_sort', 'bar_sort'],
    # values=['exp_density', 'exp_moran', 'exp_lb', 'var_density', 'var_moran', 'var_lb'],
    values=['exp_moran',],
    index=['coord', 'prob'],
    aggfunc='first'
)

summary

exp_moran                                              \
zones                    2                                               
zonal_sort            None            distance_0            projection   
bar_sort        distance_0 projection distance_0 projection distance_0   
coord   prob                                                             
cluster equal      -0.1253    -0.1276    -0.1985    -0.2043    -0.1915   
        unequal    -0.0786    -0.1228    -0.1248    -0.1248    -0.1248   
grid    equal      -0.1116    -0.2019    -0.1781    -0.1758    -0.1758   
        unequal    -0.1174    -0.0574    -0.1174    -0.1174    -0.1174   
meuse   equal      -0.2683    -0.2547    -0.2745    -0.2745    -0.2745   
        unequal    -0.1968    -0.1676    -0.2193    -0.2183    -0.2193   
random  equal      -0.1618    -0.1318    -0.1603    -0.1618    -0.1600   
        unequal    -0.0869    -0.1104    -0.1336    -0.1341    -0.1336   

                                                                        \
zones                               3                                    
zonal_sort                       None            distance_0              
bar_sort        projection distance_0 projection distance_0 projection   
coord   prob                                                             
cluster equal      -0.1985    -0.2449    -0.1455    -0.2392    -0.2384   
        unequal    -0.1248    -0.1149    -0.1299    -0.1058    -0.1294   
grid    equal      -0.1781    -0.1489    -0.1537    -0.1900    -0.1576   
        unequal    -0.1174    -0.0971    -0.0973    -0.1215    -0.1215   
meuse   equal      -0.2745    -0.2612    -0.1856    -0.3335    -0.3335   
        unequal    -0.2183    -0.1839    -0.1416    -0.2228    -0.2228   
random  equal      -0.1603    -0.1551    -0.1615    -0.1573    -0.1573   
        unequal    -0.1336    -0.0977    -0.1498    -0.1219    -0.1274   

                                                                        \
zones                                          4                         
zonal_sort      projection                  None            distance_0   
bar_sort        distance_0 projection distance_0 projection distance_0   
coord   prob                                                             
cluster equal      -0.2385    -0.2485    -0.3012    -0.2648    -0.3162   
        unequal    -0.1068    -0.1083    -0.1405    -0.0956    -0.1454   
grid    equal      -0.1708    -0.1478    -0.1527    -0.1396    -0.2041   
        unequal    -0.1229    -0.1197    -0.0938    -0.1060    -0.1224   
meuse   equal      -0.3335    -0.3335    -0.2753    -0.1998    -0.2930   
        unequal    -0.2228    -0.2228    -0.2206    -0.1668    -0.2395   
random  equal      -0.1571    -0.1568    -0.1502    -0.1106    -0.1756   
        unequal    -0.1245    -0.1245    -0.1096    -0.0926    -0.1158   

                                                  
zones                                             
zonal_sort                 projection             
bar_sort        projection distance_0 projection  
coord   prob                                      
cluster equal      -0.3118    -0.3211    -0.3240  
        unequal    -0.1450    -0.1302    -0.1267  
grid    equal      -0.1683    -0.1817    -0.1761  
        unequal    -0.1186    -0.1085    -0.1220  
meuse   equal      -0.2926    -0.3298    -0.3271  
        unequal    -0.2307    -0.2223    -0.2203  
random  equal      -0.1734    -0.1756    -0.1756  
        unequal    -0.1158    -0.1112    -0.1116

# Meuse

In [8]:
def evaluate_meuse(
        coords_dict,
        probs_dict,
        n_values=[16],
        zone_list=[(3, 3)],
        sort_method_list = [
            'lexico-xy', 'lexico-yx', 'random', 'angle_0', 'distance_0',
            'projection', 'center', 'spiral', 'max',
            'hilbert', 'farthest', 'grid', 'shell'
        ],       
        zonal_sort_list = [
            'lexico-xy', 'lexico-yx', 'random', 'angle_0', 'distance_0',
            'projection', 'center', 'spiral', 'max',
            'hilbert', 'farthest', 'grid', 'shell'
        ],
        zone_mode_list=['sweep' 'cluster'],
        tolerance=5,
        split_size=1e-3,
):
    records = []

    # pre‐compute all combinations
    combos = list(itertools.product(
        n_values,
        zone_list,
        zone_mode_list,
        zonal_sort_list,
        sort_method_list
    ))
    coords = coords_dict['meuse_unequal']
    probs = probs_dict['meuse_unequal']

    print(np.sum(probs))
    best_moran_sofar = 0

    for n, zones, zone_mode, zonal_sort, sort_method in tqdm(
        combos,
        desc="Total combos",
        unit="combo"
    ):

        # print(n, zones, zone_mode, zonal_sort, sort_method)

        modified_probs = inclusion_probabilities(probs, n=n)
        kss = gs.sampling.KMeansSpatialSamplingSimple(
            coords, modified_probs,
            n=n,
            n_zones=zones,
            tolerance=tolerance,
            split_size=split_size,
            zone_mode=zone_mode,
            sort_method=sort_method,
            zonal_sort=zonal_sort,
            max_missed_samples=1
        )

        density_expected = np.round(kss.expected_score(), 4)
        density_val = np.round(kss.var_score(), 4)

        moran_scores, lb_scores = score_all_samples_moran_lb(coords, modified_probs, kss.all_samples)

        moran_expected = np.round(kss.expected_score(moran_scores), 4)
        moran_val = np.round(kss.var_score(moran_scores), 4)

        lb_expected = np.round(kss.expected_score(lb_scores), 4)
        lb_val = np.round(kss.var_score(lb_scores), 4)

        if moran_expected < best_moran_sofar:
            best_moran_sofar = moran_expected
            print()

        records.append({
            'n': n,
            'zones': zones if zone_mode == 'cluster' else f"{zones[0]}×{zones[1]}",
            'zone_mode': zone_mode,
            'bar_sort': sort_method,
            'zonal_sort': zonal_sort if zonal_sort else 'None',
            'exp_density': density_expected,
            'exp_moran': moran_expected,
            'exp_lb': lb_expected,
            'var_density': density_val,
            'var_moran': moran_val,
            'var_lb': lb_val,
        })

    return pd.DataFrame.from_records(records)


In [9]:
df = evaluate_meuse(
    coords_dict,
    probs_dict,
    n_values=[15],
    zone_list=[(1, 1), (2, 2), (3, 3), (4, 4), (3, 4), (4, 3)],
    #zone_list=[ (3, 3)],
    sort_method_list=[
            'lexico-xy', 'lexico-yx', 'random', 'angle_0', 'distance_0',
            'projection', 'center', 'spiral', 'max',
            'hilbert', 'farthest', 'grid', 'shell'
        ],
    zonal_sort_list=[None, 'lexico-xy', 'lexico-yx', 'random', 'angle_0', 'distance_0',
            'projection', 'center', 'spiral', 'max',
            'hilbert', 'farthest', 'grid', 'shell'],
    # zonal_sort_list=[None],
    zone_mode_list=['sweep'],
    tolerance=5,
    split_size=1e-3,
)

4.0


Total combos:   0%|          | 0/1092 [00:00<?, ?combo/s]

Total combos:   0%|          | 1/1092 [00:14<4:19:27, 14.27s/combo]

Total combos:   0%|          | 5/1092 [00:58<3:31:56, 11.70s/combo]


KeyboardInterrupt: 

In [46]:
summary = df.pivot_table(
    values=['exp_density', 'exp_moran', 'exp_lb'],
    columns=['n', 'zones'],
    index=['zonal_sort', 'bar_sort'],
    aggfunc='first'
)

# 1) Temporarily
with pd.option_context('display.max_rows', None):
    display(summary)



exp_density                                          \
n                              15                                           
zones                         1×1     2×2     3×3     3×4     4×3     4×4   
zonal_sort bar_sort                                                         
None       angle_0         0.0547  0.0458  0.0526  0.0498  0.0564  0.0599   
           center          0.0812  0.0624  0.0537  0.0475  0.0472  0.0488   
           distance_0      0.0716  0.0521  0.0527  0.0528  0.0579  0.0494   
           farthest        0.0653  0.0527  0.0495  0.0445  0.0535  0.0406   
           grid            0.0635  0.0569  0.0504  0.0546  0.0566  0.0566   
           hilbert         0.0630  0.0605  0.0506  0.0505  0.0555  0.0526   
           lexico-xy       0.0691  0.0539  0.0532  0.0528  0.0561  0.0554   
           lexico-yx       0.0701  0.0482  0.0495  0.0481  0.0531  0.0513   
           max             0.0583  0.0498  0.0504  0.0421  0.0540  0.0539   
           projection      0.0781  0.0537  0.0550  0.0499  0.0525  0.0512   
           random          0.0804  0.0531  0.0518  0.0518  0.0467  0.0516   
           shell           0.0583  0.0495  0.0508  0.0477  0.0470  0.0523   
           spiral          0.0772  0.0488  0.0543  0.0495  0.0520  0.0560   
angle_0    angle_0         0.0551  0.0473  0.0496  0.0518  0.0468  0.0528   
           center          0.0777  0.0485  0.0532  0.0502  0.0555  0.0475   
           distance_0      0.0604  0.0523  0.0524  0.0517  0.0559  0.0489   
           farthest        0.0602  0.0547  0.0579  0.0474  0.0596  0.0504   
           grid            0.0563  0.0616  0.0526  0.0509  0.0564  0.0577   
           hilbert         0.0600  0.0583  0.0629  0.0592  0.0607  0.0582   
           lexico-xy       0.0692  0.0544  0.0610  0.0588  0.0580  0.0542   
           lexico-yx       0.0611  0.0594  0.0504  0.0519  0.0570  0.0485   
           max             0.0700  0.0547  0.0473  0.0603  0.0512  0.0552   
           projection      0.0688  0.0600  0.0506  0.0643  0.0574  0.0541   
           random          0.0805  0.0671  0.0580  0.0476  0.0557  0.0542   
           shell           0.0588  0.0481  0.0560  0.0516  0.0484  0.0511   
           spiral          0.0809  0.0600  0.0537  0.0517  0.0498  0.0481   
center     angle_0         0.0576  0.0677  0.0612  0.0712  0.0734  0.0722   
           center          0.0761  0.0637  0.0750  0.0762  0.0766  0.0734   
           distance_0      0.0685  0.0766  0.0742  0.0832  0.0884  0.0781   
           farthest        0.0644  0.0747  0.0731  0.0755  0.0832  0.0778   
           grid            0.0692  0.0722  0.0731  0.0822  0.0740  0.0721   
           hilbert         0.0684  0.0666  0.0758  0.0733  0.0807  0.0805   
           lexico-xy       0.0643  0.0680  0.0694  0.0835  0.0805  0.0816   
           lexico-yx       0.0669  0.0656  0.0733  0.0838  0.0726  0.0783   
           max             0.0670  0.0589  0.0684  0.0768  0.0707  0.0740   
           projection      0.0727  0.0571  0.0679  0.0734  0.0734  0.0749   
           random          0.0784  0.0614  0.0730  0.0773  0.0751  0.0759   
           shell           0.0562  0.0538  0.0728  0.0683  0.0702  0.0687   
           spiral          0.0753  0.0571  0.0817  0.0816  0.0683  0.0806   
distance_0 angle_0         0.0661  0.0661  0.0680  0.0677  0.0579  0.0752   
           center          0.0884  0.0715  0.0667  0.0616  0.0728  0.0676   
           distance_0      0.0640  0.0685  0.0678  0.0566  0.0615  0.0798   
           farthest        0.0666  0.0684  0.0743  0.0572  0.0662  0.0681   
           grid            0.0677  0.0772  0.0742  0.0767  0.0712  0.0655   
           hilbert         0.0630  0.0664  0.0640  0.0674  0.0691  0.0710   
           lexico-xy       0.0605  0.0652  0.0743  0.0764  0.0671  0.0651   
           lexico-yx       0.0649  0.0733  0.0678  0.0676  0.0605  0.0707   
           max             0.0729  0.0640  0.0628  0.0559  0.0615  0.0714   
           project

In [47]:
top_n_records(df, ['zonal_sort', 'bar_sort', 'zones'], k=10)

,name,exp_moran
0,shell - farthest - 2×2,-0.3117
1,lexico-yx - projection - 1×1,-0.3096
2,hilbert - projection - 1×1,-0.3061
3,None - farthest - 4×4,-0.3060
4,distance_0 - farthest - 2×2,-0.3057
5,projection - projection - 4×3,-0.3046
6,angle_0 - distance_0 - 1×1,-0.3041
7,distance_0 - distance_0 - 1×1,-0.3038
8,projection - random - 4×3,-0.3038
9,projection - max - 4×3,-0.3038


In [ ]:
def evaluate_meuse_loop(
        coords_dict,
        probs_dict,
        num_loops=10,
        n_values=[4],
        zone_list=[(1, 1), (2, 2), (3, 3)],
        sort_method_list=[
            'lexico-xy', 'lexico-yx', 'random', 'angle_0', 'distance_0',
            'projection', 'center', 'spiral', 'max',
            'hilbert', 'farthest', 'grid', 'shell'
        ],
        zonal_sort_list=[
            'lexico-xy', 'lexico-yx', 'random', 'angle_0', 'distance_0',
            'projection', 'center', 'spiral', 'max',
            'hilbert', 'farthest', 'grid', 'shell'
        ],
        zone_mode_list=['sweep' 'cluster'],
        tolerance=5,
        split_size=1e-3,
):
    records = []

    # pre‐compute all combinations
    combos = list(itertools.product(
        n_values,
        zone_list,
        zone_mode_list,
        zonal_sort_list,
        sort_method_list
    ))
    coords = coords_dict['meuse_unequal']
    probs = probs_dict['meuse_unequal']

    best_moran_sofar = 0

    for _ in tqdm(range(num_loops), desc="Loops", unit="loop"):
        for n, zones, zone_mode, zonal_sort, sort_method in combos:
        # tqdm(
        #     combos,
        #     desc="Total combos",
        #     unit="combo"
        # ):

            # print(n, zones, zone_mode, zonal_sort, sort_method)

            modified_probs = inclusion_probabilities(probs, n=n)
            kss = gs.sampling.KMeansSpatialSamplingSimple(
                coords, modified_probs,
                n=n,
                n_zones=zones,
                tolerance=tolerance,
                split_size=split_size,
                zone_mode=zone_mode,
                sort_method=sort_method,
                zonal_sort=zonal_sort,
                max_missed_samples=1
            )

            density_expected = np.round(kss.expected_score(), 4)
            density_val = np.round(kss.var_score(), 4)

            moran_scores, lb_scores = score_all_samples_moran_lb(coords, modified_probs, kss.all_samples)

            moran_expected = np.round(kss.expected_score(moran_scores), 4)
            moran_val = np.round(kss.var_score(moran_scores), 4)

            lb_expected = np.round(kss.expected_score(lb_scores), 4)
            lb_val = np.round(kss.var_score(lb_scores), 4)

            if moran_expected < best_moran_sofar:
                best_moran_sofar = moran_expected
                print('\n====================================')
                print('A NEW BEST FOUND')
                print(f'Moran score: {moran_expected}' )
                print(f'zone_mode: {zone_mode}')
                print(f'zonal_sort: {zonal_sort}')
                print(f'bar_sort: {sort_method}')
                print(f'zones: {zones}')
            
            cluster_membership = tuple(kss.cluster_membership)  # Or .tolist() or .copy()
            records.append({
                'n': n,
                'zones': zones if zone_mode == 'cluster' else f"{zones[0]}×{zones[1]}",
                'zone_mode': zone_mode,
                'bar_sort': sort_method,
                'zonal_sort': zonal_sort if zonal_sort else 'None',
                'exp_density': density_expected,
                'exp_moran': moran_expected,
                'exp_lb': lb_expected,
                'var_density': density_val,
                'var_moran': moran_val,
                'var_lb': lb_val,
                'cluster_membership': cluster_membership,

            })

    return pd.DataFrame.from_records(records)


In [ ]:
df = evaluate_meuse_loop(
    coords_dict,
    probs_dict,
    num_loops=500,
    n_values=[15],
    # zone_list=[(1, 1), (2, 2)],#, (2, 1), (1, 2), (2, 3), (3, 2)],
    zone_list=[ (1, 1),],
    # zone_list=[12],
    sort_method_list=['farthest', 'hilbert'],
    #zonal_sort_list=[None, 'random', 'projection'],
    # zonal_sort_list=['shell', 'hilbert'],
    zonal_sort_list=[None],
    zone_mode_list=['sweep'],
    # zone_mode_list=['cluster'],
    tolerance=5,
    split_size=1e-3,
)

Loops:   0%|          | 0/500 [00:00<?, ?loop/s]


A NEW BEST FOUND
Moran score: -0.2166
zone_mode: sweep
zonal_sort: None
bar_sort: farthest
zones: (1, 1)


Loops:   0%|          | 1/500 [00:26<3:41:35, 26.64s/loop]


A NEW BEST FOUND
Moran score: -0.2591
zone_mode: sweep
zonal_sort: None
bar_sort: hilbert
zones: (1, 1)


Loops:   1%|          | 3/500 [01:17<3:37:55, 26.31s/loop]

...

Loops:   9%|▉         | 46/500 [15:45<2:13:21, 17.63s/loop]


A NEW BEST FOUND
Moran score: -0.296
zone_mode: sweep
zonal_sort: None
bar_sort: hilbert
zones: (1, 1)


Loops:  11%|█▏        | 57/500 [19:04<2:31:26, 20.51s/loop]


A NEW BEST FOUND
Moran score: -0.3059
zone_mode: sweep
zonal_sort: None
bar_sort: hilbert
zones: (1, 1)


Loops:  13%|█▎        | 65/500 [21:52<2:29:46, 20.66s/loop]

In [34]:
summary = df.pivot_table(
    columns=['sort'],
    values=['exp_density', 'exp_moran', 'var_density', 'var_moran'],
    index=['zones'],
    aggfunc='first'
)

summary

KeyError: 'sort'